In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

---

## 1. Price parsing

- `price_original`: Giá gốc (giá chưa giảm, đơn vị: VND)
- `price_discounted`: Giá khuyến mãi (giá đã giảm, đơn vị: VND)
- `precent_discount`: Phần trăm số tiền đã giảm (đơn vị: %)

<pre>
- Có giá khuyến mãi: 
    &lt;fare-sale&gt; chứa giá khuyến mãi
        &lt;fareSmall&gt; chứa giá gốc và phần trăm ưu đãi
- Không có giá khuyến mãi:
    &lt;fare&gt; chứa giá gốc
        &lt;fareSmall&gt; không chứa dữ liệu
</pre>

In [ ]:
def is_regular_fare(block) -> bool:
    """
    Kiểm tra chuyến xe có khuyến mãi hay không.

    Returns:
        True  -> không có giá khuyến mãi (fare)
        False -> có giá khuyến mãi (fare-sale)
    """
    fare = block.find("div", class_="fare")
    return bool(fare)


def parse_regular_fare(block):
    """
    Trích xuất giá vé cho chuyến không có khuyến mãi.

    Returns:
        dict: {
            "price_original": str,  # Giá gốc
            "price_discounted": None,
            "percent_discount": None
        }
    """
    fare = block.find("div", class_="fare")
    price_original = (
        fare.get_text(strip=True)
        if fare else None
    )

    return {
        "price_original": price_original,
        "price_discounted": None,
        "percent_discount": None,
    }


def parse_discount_metadata(block):
    """
    Trích xuất giá gốc và phần trăm giảm giá từ thẻ 'fareSmall'.

    Returns:
        tuple: (price_original, percent_discount)
    """
    fare_small = block.find("div", class_="fareSmall")
    price_original = (
        fare_small.find("div", class_="small").get_text(strip=True)
        if fare_small else None
    )

    try:
        percent_discount = (
            fare_small.find("div", class_="percent").get_text(strip=True)
            if fare_small and fare_small.find("div", class_="percent")
            else None
        )
    except Exception:
        percent_discount = None

    return price_original, percent_discount


def parse_discounted_fare(block):
    """
    Trích xuất giá vé cho chuyến có khuyến mãi.

    Returns:
        dict: {
            "price_original": str,   # Giá gốc
            "price_discounted": str, # Giá sau giảm
            "percent_discount": str  # Phần trăm giảm
        }
    """
    price_original, percent_discount = parse_discount_metadata(block)

    fare_sale = block.find("div", class_="fare-sale")
    price_discounted = (
        fare_sale.get_text(strip=True).strip() if fare_sale and fare_sale.get_text(strip=True) else None
    )

    return {
        "price_original": price_original,
        "price_discounted": price_discounted,
        "percent_discount": percent_discount,
    }


In [ ]:
def parse_price(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe \n
    Trả về: giá gốc, giá khuyến mãi (nếu có), phần trăm khuyến mãi (nếu có)
    '''
    
    if is_regular_fare(block):
        return parse_regular_fare(block)
    else:
        return parse_discounted_fare(block)

---

## 2. Bus info parsing

In [4]:
def parser_trip_bus_info(container):
    '''
    Trích xuất thông tin từ một container chứa thông tin chuyến đi. \n
    Trả về Tuple: tên nhà xe, đánh giá nhà xe, loại ghế.
    '''
    
    # bus name / company name
    bus_element = container.find('div', class_='bus-name')
    company_name = bus_element.get_text(strip=True) if bus_element else None

    # bus rating
    rating_element = container.find('div', class_='bus-rating').find('span')
    bus_rating = rating_element.get_text(strip=True) if rating_element else None

    # seat_type
    seat_type = container.find('div', class_='seat-type')
    seat_type = seat_type.get_text(strip=True) if seat_type else None

    return {
        'company_name': company_name,
        'bus_rating': bus_rating,
        'seat_type': seat_type
    }

---

## 3. Route parsing

In [5]:
# Dữ liệu này nằm ở ô filter chuyến đi

def parse_route_info(block):
    '''
    Trích xuất dữ liệu từ filter của trang web \n
    Trả về: ngày khởi hành, nơi xuất phát (thành phố hiện tại), nơi đến (nơi đặt vé đến)
    '''

    departure_date, start_point, destination = None, None, None

    try:
        departure_date = block.find('p', class_='date-input-value').get_text(strip=True)
        start_point = block.find(id="from_input").get('value')
        destination = block.find(id="to_input").get('value')
    except Exception:
        pass

    return {
        'departure_date': departure_date,
        'start_point': start_point,
        'destination': destination
    }

---

## 4. Details trip info

### 4.1 Departure

In [6]:
# Dữ liệu này nằm trong container > 'from_content'

def parse_departure_trip_info(from_content):
    """
    Trích xuất thông tin điểm đi từ một container 'from_content'.
    Trả về một tuple chứa: giờ khởi hành, địa điểm đón khách.
    """
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not from_content:
        return None, None

    # departure time
    departure_time_element = from_content.find('div', class_='hour')
    departure_time = departure_time_element.get_text(strip=True) if departure_time_element else None

    # departure place
    from_place_tag = from_content.find('div', class_='place')
    pickup_point = from_place_tag.get_text(strip=True) if from_place_tag else None
    
    return {
        'departure_time': departure_time,
        'pickup_point': pickup_point
    }

### 4.2 Arrival

In [7]:
def parse_arrival_trip_info(to_content):
    """
    Trích xuất thông tin điểm đến từ một container 'to_content'.\n
    Trả về một tuple chứa: ngày đến, thời gian đến, điểm trả khách.
    """
    
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not to_content:
        return None, None, None

    # lấy ngày đến
    date_arrival_tag = to_content.find('span', class_="text-date-arrival-time")
    arrival_date = date_arrival_tag.get_text(strip=True) if date_arrival_tag else None
    
    
    # lấy giờ và địa điểm trả khách
    content_to_info = to_content.find('div', class_='content-to-info')
    if content_to_info:
        to_hour_tag = content_to_info.find('div', class_='hour')
        arrival_time = to_hour_tag.get_text(strip=True) if to_hour_tag else None
        
        dropoff_place_element = content_to_info.find('div', class_='place')
        dropoff_point = dropoff_place_element.get_text(strip=True) if dropoff_place_element else None
        
    return {
        'arrival_date': arrival_date,
        'arrival_time': arrival_time,
        'dropoff_point': dropoff_point
    }


---

In [8]:
def parse_trip_timing(container):
    """
    Trích xuất thông tin chi tiết về chuyến đi (giờ, nơi đi - đến, thời gian di chuyển). \n
    Trả về: dict chứa thông tin khởi hành, điểm đến và thời lượng chuyến.
    """
    
    # Tìm khối chứa thông tin đi và đến
    from_to_content = container.find('div', class_="from-to-content")

    # Nếu không tìm thấy, trả về dict rỗng có cấu trúc sẵn
    if not from_to_content:
        return {
            "duration": None,
            "departure_time": None,
            "pickup_point": None,
            "departure_date": None,
            "arrival_date": None,
            "arrival_time": None,
            "dropoff_point": None,
        }

    # Lấy thông tin nơi khởi hành
    from_content = from_to_content.find('div', class_='content from')
    dict_departure_info = parse_departure_trip_info(from_content)

    # Lấy thông tin nơi đến
    to_content = from_to_content.find('div', class_='content to')
    dict_arrival_info = parse_arrival_trip_info(to_content)

    # Lấy thời gian di chuyển
    duration_tag = from_to_content.find('div', class_="duration")
    duration = duration_tag.get_text(strip=True) if duration_tag else None

    # Gộp toàn bộ thông tin lại
    trip_data = dict_departure_info | dict_arrival_info | {'duration': duration}
    
    return trip_data


In [9]:
def compile_trip_info(block):
    '''
    Tập hợp toàn bộ thông tin của 1 chuyến xe từ 1 khối dữ liệu (block). \n
    Trả về: dict chứa thông tin xe, lịch trình và giá vé.
    '''
    
    # Lấy thông tin chính của nhà xe
    dict_bus_info = parser_trip_bus_info(block)
    
    # Lấy thông tin giờ đi - giờ đến, điểm đón - trả
    dict_trip_details = parse_trip_timing(block)
    
    # Lấy thông tin giá vé (giá gốc, giá khuyến mãi)
    dict_price = parse_price(block)
    
    # Gộp tất cả dữ liệu vào 1 dictionary duy nhất
    trip_data = dict_bus_info | dict_trip_details | dict_price
    
    return trip_data


---

## Parsing rating

In [10]:
def parse_ratings_from_container(container):
    '''
    Trích xuất thông tin đánh giá (rating) của từng nhà xe trong 1 container. \n
    Trả về: list các cặp (rate_title, rate_point) hoặc [(None, None)] nếu không có dữ liệu.
    '''
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')   # Tìm tất cả khối chứa thông tin đánh giá
        
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')   # Mỗi phần tử chứa tiêu đề và điểm
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)   # Tiêu đề đánh giá
                rate_point = rate_ps[1].get_text(strip=True)   # Điểm đánh giá
                ratings.append((rate_title, rate_point))       # Lưu vào danh sách
        
        if ratings:
            return ratings     # Trả về danh sách nếu có dữ liệu
        else:
            return [(None, None)]   # Không có dữ liệu đánh giá
            
    except Exception:
        return [(None, None)]   # Trường hợp lỗi vẫn trả về giá trị mặc định


total

In [11]:
def extract_all_trips(soup):
    '''
    Trích xuất và gộp thông tin chuyến xe, tuyến đường và đánh giá nhà xe thành một DataFrame duy nhất.
    Trả về: DataFrame chứa toàn bộ dữ liệu chuyến xe.
    '''
    dict_route = parse_route_info(soup)  # Lấy thông tin tuyến đường (đi - đến)
    containers = soup.find_all("div", class_="container")  # Tìm tất cả container chứa chuyến xe

    lst_trips_info = []

    for container in containers:
        # ---- Lấy thông tin chuyến xe ----
        dict_trip_info = compile_trip_info(container) | dict_route

        # ---- Lấy thông tin đánh giá ----
        ratings = parse_ratings_from_container(container)

        # ratings là list các tuple [(rate_title, rate_point), ...]
        # ta chuyển nó thành dict dạng {'rating_TênTiêuĐề': điểm}
        rating_dict = {
            f"{title}": point
            for title, point in ratings
            if title is not None and point is not None
        }

        # ---- Gộp thông tin chuyến xe và rating ----
        full_info = dict_trip_info | rating_dict

        # ---- Đưa vào DataFrame ----
        df_trip_info = pd.DataFrame([full_info])
        lst_trips_info.append(df_trip_info)

    # ---- Gộp toàn bộ chuyến xe lại thành 1 dataframe ----
    all_trips_info = pd.concat(lst_trips_info, ignore_index=True)

    return all_trips_info


---

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 5. Handle Button logic

### 5.1. Button `Xem thêm chuyến`

In [13]:
def click_load_more(driver, max_click=7, max_wait=15):
    """
    Click nút 'Xem thêm chuyến' an toàn, tối đa max_click lần.
    - Dùng JS click để tránh lỗi bị che.
    - Tự dừng nếu không thấy container mới xuất hiện.
    """
    wait = WebDriverWait(driver, max_wait)
    click_count = 0
    fail_count = 0
    MAX_FAIL = 3

    for _ in range(max_click):
        try:
            containers_before = len(driver.find_elements(By.CLASS_NAME, "container"))
            btn = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "load-more")))

            if not btn.is_displayed() or not btn.is_enabled():
                print("Nút 'Xem thêm chuyến' không khả dụng.")
                break

            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            time.sleep(random.uniform(0.8, 1.5))
            driver.execute_script("arguments[0].click();", btn)
            click_count += 1

            time.sleep(random.uniform(2.0, 3.0))

            try:
                WebDriverWait(driver, 10).until(
                    lambda d: len(d.find_elements(By.CLASS_NAME, "container")) > containers_before
                )
                fail_count = 0
            except TimeoutException:
                fail_count += 1
                if fail_count >= MAX_FAIL:
                    print("Không thấy container mới sau nhiều lần → dừng.")
                    break

        except (TimeoutException, NoSuchElementException):
            print("Không còn nút 'Xem thêm chuyến' → dừng.")
            break
        except StaleElementReferenceException:
            time.sleep(1)
            continue
        except Exception:
            fail_count += 1
            if fail_count >= MAX_FAIL:
                break
            time.sleep(2)
            continue

    print(f"Hoàn tất — đã click {click_count} lần.")
    return click_count

### 5.2. Button `Xem các đánh giá`

In [14]:
def expand_ratings(driver, click_prob=0.65, delay_range=(0.5, 1.2)):
    """
    Mở ngẫu nhiên các phần đánh giá (rating) trên trang Vexere.
    - Không giới hạn số lần click.
    - Random bỏ qua một số nút để giả lập hành vi người thật.
    """
    try:
        time.sleep(2)
        stars = driver.find_elements(By.CLASS_NAME, "bus-rating-button")
        if not stars:
            print("Không tìm thấy nút đánh giá.")
            return 0

        total = len(stars)
        clicks = 0

        for i in range(total):
            if random.random() > click_prob:
                continue

            try:
                stars = driver.find_elements(By.CLASS_NAME, "bus-rating-button")
                if i >= len(stars):
                    break
                star = stars[i]

                if not star.is_displayed():
                    continue

                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
                time.sleep(random.uniform(*delay_range))
                driver.execute_script("arguments[0].click();", star)
                clicks += 1
                time.sleep(random.uniform(*delay_range))

            except StaleElementReferenceException:
                continue
            except Exception:
                continue

        print(f"Đã click {clicks}/{total} nút đánh giá.")
        return clicks

    except Exception:
        print("Không thể mở phần đánh giá.")
        return 0


### 5.3. Button `Tìm kiếm`

In [15]:
def click_search(driver, retries=3):
    """
    Click vào nút tìm kiếm trên trang Vexere (an toàn, tự scroll, retry nếu lỗi).
    """
    wait = WebDriverWait(driver, 10)
    
    for _ in range(retries):
        try:
            btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "button-search")))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", btn)
            print("Click vào nút tìm kiếm thành công")
            return True
        except Exception:
            print("Click tìm kiếm lỗi, thử lại")
            time.sleep(1)

    print("Không thể click vào nút tìm kiếm")
    return False

## 6. Automate the process of filtering website data

In [16]:
def get_target_date_components(days=0):
    """
    Trả về ngày và tháng-năm mục tiêu cách hiện tại `days` ngày.

    Parameters
    ----------
    days : int, optional
        Số ngày cộng thêm từ ngày hiện tại (mặc định = 0).

    Returns
    -------
    dict
        {'day': '15', 'month_year': '10-2025'}
    """
    # Ngày mục tiêu = ngày hiện tại + days ngày
    target_date = datetime.today() + timedelta(days=days)
    month_year = f"{target_date.month:02d}-{target_date.year}"
    day = str(target_date.day)

    return {
        'day': day,
        'month_year': month_year
    }

In [17]:
def set_search_filters(driver, start_city: str, destination_city: str, days=0):
    """
    Chọn điểm đi, điểm đến và ngày khởi hành trên trang Vexere.

    Parameters
    ----------
    driver : webdriver
        Đối tượng Selenium WebDriver đang điều khiển trình duyệt.
    start_city : str
        Tên thành phố khởi hành.
    destination_city : str
        Tên thành phố điểm đến.
    days : int, optional
        Số ngày tính từ hôm nay để chọn ngày đi (mặc định = 0).

    Returns
    -------
    bool
        True nếu chọn ngày thành công, False nếu xảy ra lỗi.
    """

    wait = WebDriverWait(driver, 10)

    try:
        # Nhập nơi đi và nơi đến
        departure_input = wait.until(EC.presence_of_element_located((By.ID, 'from_input')))
        destination_input = wait.until(EC.presence_of_element_located((By.ID, 'to_input')))

        departure_input.clear()
        destination_input.clear()

        departure_input.send_keys(start_city)
        destination_input.send_keys(destination_city)
        time.sleep(0.5)

        # Mở phần chọn ngày đi
        date_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "departure-date-select")))
        date_btn.click()
        time.sleep(1)

        # Lấy thông tin ngày và tháng cần chọn
        target = get_target_date_components(days)
        target_day = target['day']
        target_month = target['month_year']

        # Tìm khối tháng (có thể dùng '-' hoặc '_')
        try:
            month_section = driver.find_element(By.ID, target_month)
        except:
            month_section = driver.find_element(By.ID, target_month.replace('-', '_'))

        # Tìm tất cả các phần tử ngày trong tháng
        day_elements = month_section.find_elements(By.CSS_SELECTOR, "p.day")

        for day in day_elements:
            if day.text.strip() == target_day:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", day)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", day)
                return True

        print("Không tìm thấy ngày cần chọn.")
        return False

    except Exception as e:
        print(f"Lỗi khi chọn bộ lọc tìm kiếm: {e}")
        return False

# FLOW OFFICIAL

---

## Crawl rating data

# -- Main --

In [18]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


In [19]:
# URL = 'https://vexere.com/'

# # Danh sách tuyến
# arrivals_HaNoi = ['Hà Giang','Quảng Ninh','Thanh Hóa','SaPa','Ninh Bình']#['Hải Phòng','Nghệ An','Sơn La']
# arrivals_SaiGon = ['Gia Lai','Bình Thuận','Ninh Thuận','Đắk Lắk','Phú Yên','Nha Trang','Bà Rịa-Vũng Tàu']


# departure_city = 'Hà Nội'
# days_offset = 1 #int(input(""))

# # Lấy ngày / tháng cần crawl
# target_date = get_target_date_components(days_offset)
# day = target_date['day']
# month_year = target_date['month_year']
# month_years = month_year.replace('-', '_')

# # Crawl từng tuyến
# for arrival_city in arrivals_HaNoi:

#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     driver.get(URL)

#     print(f"Đang crawl: {departure_city} → {arrival_city}")

#     filter_success = set_search_filters(
#         driver,
#         start_city=departure_city,
#         destination_city=arrival_city,
#         days=days_offset
#     )

#     if not filter_success:
#         print(f"Không thể chọn ngày cho {arrival_city}")
#         continue

#     time.sleep(2)
#     click_search(driver)    # Click tìm kiếm sau khi lọc dữ liệu
#     time.sleep(2.5)
#     click_load_more(driver,5) # Click xem thêm để thấy đc nhiều chuyến xe
    
#     expand_ratings(driver, 0.6)  # Mở các thông tin đánh giá từng chuyến
#     time.sleep(1.5)

#     # Parse dữ liệu HTML
#     soup = BeautifulSoup(driver.page_source, "html.parser")

#     try:
#         df_trips_info = extract_all_trips(soup)

#         save_path = f"../../data/processed/{departure_city}_{arrival_city}_{day}_{month_years}.csv"
#         df_trips_info.to_csv(save_path, index=False, encoding='utf-8')

#         print(f"Lấy dữ liệu {arrival_city} thành công! → {save_path}")

#     except Exception as e:
#         html_path = f"../../data/site/{departure_city}_{arrival_city}_{day}_{month_years}.html"
#         with open(html_path, 'w', encoding='utf-8') as f:
#             f.write(soup.prettify()) 
#         print(f"Lưu HTML để debug: {html_path}\n{e}")

# # Đóng trình duyệt sau khi hoàn tất
#     driver.quit()

In [ ]:
import time, random, os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver as uc

# ====================== CẤU HÌNH BAN ĐẦU ======================
URL = 'https://vexere.com/'

# Danh sách tuyến
arrivals_HaNoi = ['Hà Giang', 'Quảng Ninh', 'Thanh Hóa', 'SaPa', 'Ninh Bình']
# arrivals_SaiGon = ['Gia Lai','Bình Thuận','Ninh Thuận','Đắk Lắk','Phú Yên','Nha Trang','Bà Rịa-Vũng Tàu']

departure_city = 'Hà Nội'
days_offset = 1  # int(input("Nhập số ngày muốn crawl: "))

# Lấy ngày / tháng cần crawl
target_date = get_target_date_components(days_offset)
day = target_date['day']
month_year = target_date['month_year']
month_years = month_year.replace('-', '_')

print(f"\n🚦 Bắt đầu crawl từ {departure_city} cho ngày: {day}/{month_year}")

# ====================== VÒNG LẶP CHÍNH ======================
for arrival_city in arrivals_HaNoi:

    # --- Cấu hình Chrome tối ưu ---
    options = Options()
    # options.add_argument("--headless=new")                   # chạy nền, không mở giao diện
    options.add_argument("--no-sandbox")                     # giảm overhead, đặc biệt khi chạy server
    options.add_argument("--disable-dev-shm-usage")          # tránh crash khi RAM thấp
    options.add_argument("--disable-gpu")                    # tắt GPU render
    options.add_argument("--disable-software-rasterizer")    # bỏ vẽ 3D
    options.add_argument("--disable-extensions")             # tắt extension không cần thiết
    options.add_argument("--disable-infobars")               # tắt banner “Chrome is being controlled...”
    options.add_argument("--disable-popup-blocking")         # tránh popup
    options.add_argument("--disable-notifications")          # tắt thông báo
    options.add_argument("--disable-blink-features=AutomationControlled")  # giảm phát hiện bot
    options.add_argument("--window-size=1920,1080")          # cố định viewport
    options.add_argument("--start-maximized")                # tối ưu hiển thị
    options.add_argument("--log-level=3")                    # giảm log
    options.add_argument("--disable-logging")                # tắt ghi log nội bộ Chrome
    options.add_argument(f"--remote-debugging-port={random.randint(9000,9999)}")  # tránh trùng port

    # --- Tắt tải ảnh, popup, plugin để tiết kiệm tài nguyên ---
    prefs = {
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2,
        "profile.managed_default_content_settings.stylesheets": 1,
        "profile.managed_default_content_settings.cookies": 1,
        "profile.managed_default_content_settings.plugins": 2,
        "profile.managed_default_content_settings.popups": 2
    }
    options.add_experimental_option("prefs", prefs)
    options.page_load_strategy = "eager"  # chỉ đợi DOM, không chờ toàn bộ page load

    # --- Khởi tạo service chỉ 1 lần ---
    service = Service(ChromeDriverManager().install())

    print(f"\n🚍 Đang crawl: {departure_city} → {arrival_city}")

    driver = None
    try:
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(URL)

        # 1️⃣ Thiết lập filter
        filter_success = set_search_filters(
            driver,
            start_city=departure_city,
            destination_city=arrival_city,
            days=days_offset
        )

        if not filter_success:
            print(f"⚠️ Không thể chọn ngày cho {arrival_city}")
            continue

        # 2️⃣ Click tìm kiếm
        time.sleep(1)
        click_search(driver)

        # 3️⃣ Chờ trang kết quả
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".bus-item"))
            )
        except TimeoutException:
            print(f"⏰ Không tìm thấy chuyến xe nào cho tuyến {arrival_city}")
            continue

        # 4️⃣ Click “Xem thêm” và mở đánh giá
        click_load_more(driver, max_click=5)
        expand_ratings(driver, click_prob=0.75)
        time.sleep(1.5)

        # 5️⃣ Parse HTML và lưu file CSV
        soup = BeautifulSoup(driver.page_source, "html.parser")
        try:
            df_trips_info = extract_all_trips(soup)

            os.makedirs("../../data/processed", exist_ok=True)
            save_path = f"../../data/processed/{departure_city}_{arrival_city}_{day}_{month_years}.csv"
            df_trips_info.to_csv(save_path, index=False, encoding='utf-8')

            print(f"✅ Lấy dữ liệu {arrival_city} thành công! → {save_path}")

        except Exception as e_parse:
            os.makedirs("../../data/site", exist_ok=True)
            html_path = f"../../data/site/{departure_city}_{arrival_city}_{day}_{month_years}_error.html"
            with open(html_path, 'w', encoding='utf-8') as f:
                f.write(soup.prettify())
            print(f"❌ Lỗi xử lý dữ liệu: {e_parse}\nĐã lưu HTML để debug: {html_path}")

        finally:
            del soup

    except WebDriverException as e_driver:
        print(f"💥 Lỗi driver khi xử lý {arrival_city}: {e_driver}")

    finally:
        if driver:
            driver.quit()
        print(f"🧹 Đã đóng driver cho tuyến {arrival_city}")
        time.sleep(random.uniform(5, 10))  # nghỉ ngẫu nhiên để tránh bị chặn IP

print("\n🎯 --- HOÀN TẤT TOÀN BỘ QUÁ TRÌNH CRAWL ---")



🚦 Bắt đầu crawl từ Hà Nội cho ngày: 1/11-2025

🚍 Đang crawl: Hà Nội → Hà Giang
💥 Lỗi driver khi xử lý Hà Giang: Message: session not created: cannot connect to chrome at 127.0.0.1:64886
from session not created: This version of ChromeDriver only supports Chrome version 142
Current browser version is 141.0.7390.123; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x4b58c3
	0x4b5904
	0x2ce76d
	0x30b20d
	0x30a159
	0x30008f
	0x2ffeb6
	0x348993
	0x34830a
	0x33c766
	0x30dac0
	0x30ede4
	0x737974
	0x732bea
	0x4de5b4
	0x4cdd28
	0x4d4d8d
	0x4bded8
	0x4be09c
	0x4a7d1a
	0x767e5d49
	0x77c0d6db
	0x77c0d661

🧹 Đã đóng driver cho tuyến Hà Giang

🚍 Đang crawl: Hà Nội → Quảng Ninh
💥 Lỗi driver khi xử lý Quảng Ninh: Message: session not created: cannot connect to chrome at 127.0.0.1:60556
from session not created: This version of ChromeDriver

KeyboardInterrupt: 